# Développement et déploiement d'un projet d'analyse de sentiment

## I) Le projet

*Air Paradis* souhaite pouvoir disposer d'un **produit IA** permettant d'**anticiper les bad buzz** sur les **réseaux sociaux**.

Le projet a alors consisté à :
- charger un jeu de données contenant des tweets étiquettés 
- réaliser l'**analyse exploratoire (*EDA*)** sur ce jeu de données pour décider quels traitements seraient opportuns
- mettre en oeuvre le **preprocessing** adéquat et surtout adapté au modèle testé
- tester différents modèles et les optimiser
- sélectionner un modèle gagnant
- comprendre le modèle grâce à l'**interprétabilité**
- rendre disponible ce modèle via une **API**
- et créer une **UI** permettant d'y accéder

## II) La démarche MLOps mise en oeuvre

L'autre facette du projet consistait à le construire en suivant les concepts d'une démarche **MLops**.

### II.1) Qu'est-ce que le *MLOps* ?

<img src="MLOPS_diagram.png" alt="MLOPS_diagram" width="1000" class="center"/>

**MLOps** est un ensemble de pratiques qui **combine** les techniques de développement logiciel (**DevOps**) et d’apprentissage automatique (**Machine Learning**) pour optimiser et automatiser le cycle de vie de l’apprentissage machine. Le but de MLOps est d’aider les organisations à **construire, déployer et gérer** des modèles d’apprentissage machine en environnement de production **plus efficacement et plus rapidement**. 

Voici quelques principes clés du MLOps :
- Développement du **pipeline de traitement des données** :
    - cela comprend le travail préliminaire de collecte des données, d'exploration, etc.
    - nettoyage, traitements, feature engineering, etc
- Développement du **pipeline de modélisation** :
    - entraînement de différents modèles, avec différentes combinaisons d'hyperparamètres
    - évaluation et sélection du meilleur modèle
- **Intégration continue** du pipeline général :
    - utilisation d'une platefrome de développement collaboratif et de gestion de version
    - pour travailler sur le code source et suivre ses modifications
    - tests unitaires, tests de performance, etc.
- **Déploiement continue** du pipeline général :
    - automatisation totale du pipeline
    - y compris collecte, préparation, recherche d'hyperparamètres
- **Tracking** :
    - utilisation de solutions (comme `MLflow`) pour faire un suivi automatisé des différents expérimentations conduites avec le pipeline
    - enregistrement des paramètres de préparation, du type de modèle, des hyperparamètres et enfin des mesures d'évaluation associées
- **Enregistrement des modèles** :
    - chaque version du modèle est consignée
    - avec toutes les données/informations ayant servi à la construire
- **Déploiement continue du modèle**
    - mettre en place un serveur ou utiliser une plateform cloud
    - y installer les dépendances nécessaires pour utiliser le modèle
    - créer une API permettant d'accéder au modèle et d'optenir des prédictions
- **Monitoring** :
    - suivi des performances du modèle en production
    - envoi d'alertes si le modèle n'est plus aussi bon sur les données en temps réel
- **Entraînement continue** :
    - en cas de dégradation de la performance (alerte)
    - mise à jour du modèle en utilisant le pipeline général automatisé
- **Gouvernance et Conformité** :
    - respecter les règlements et exigences relatifs à la confidentialité des données,
    - ainsi qu'à la sécurité et à l’utilisation éthique de l’IA
- **Scalabilité** :
    - absorber les évolutions de la demande
    - système distribué pour permettre les mutations des solutions logiciels et matériels 
- **Collaboration et documentation** :
    - documenter l'ensemble du cycle de vie du produit IA (le code, la donnée, l'architecture)
    - communiquer / évangéliser pour améliorer les performances et détecter les problèmes
 
Pour chacune de ces étapes, des solutions logiciels existent et permettent d'implémenter des projets IA avec une meilleure efficacité ainsi qu'une plus grand fiabilité. Ils ne sont pas tous obligatoires et dépendent des besoins de chaque projet.


### II.2) Et pour notre cas ?

Pour notre projet nous avons appliqué certaines facettes du *MLOps* :
- utilisation de **`MLFlow`** pour :
    - tracker les hyperparamètres de nos différents modèles
    - tracker les métriques
    - enregistrer les modèles que nous souhaitons conserver
    - charger le modèle gagnant et faire des prédictions
    - tester le service de *serving*
- mis en oeuvre de **pipelines de preprocessing et/ou apprentissage** suivant les cas
- utilisation des ces pipelines pour **optimiser les hyperparamètres** avec **`Optuna`**
- **intégration continue du code** avec **`GitHub`**
- mis en oeuvre d'un **pipeline de déploiement continu** de l'API et de l'UI (grâce à **AZURE** et **GitHub Actions**)

## III) Le preprocessing

Pour le preprocessing, nous avions plusieurs cas :
- les **opérations générales** pouvant être appliquées aussi bien pour notre modèle simple (basé sur une vectorisation de type bag-of-words) que pour notre modèle avancé (basé sur une vectorisation de type word embedding) :
    - mettre en minuscule
    - retirer les codes HTML correspondants à des caractères spéciaux
    - remplacer (par "url" par exemple) les liens hypertextes
    - remplacer (par "mail" par exemple) les adresses mails
    - retirer les séquences d'échappement
    - corriger les mots avec 3+ lettres répétées
    - corriger les contractions
    - remplacer les emoticons avec leur sens
    - remplacer (par "hashtag" par exemple) les #xxxx
    - remplacer (par "mention" par exemple) les @xxxx
    - enfin appliquer les méthode `strip`
- les opérations **plus spécifiques** à un modèle :
    - **modèle simple Machine Learning** :
        - mettre en minuscule (si ce n'est pas déjà fait)
        - retirer les stopwords
        - retirer d'éventuels stopwords personnalisés
        - retirer la ponctuation
        - retirer les espaces en trop
        - retirer les nombres et affiliés (exemple : "nine")
        - filtrer sur les Part-of-Speech tags (exemple : ne garder que les adjectifs)
        - normaliser via stemmatisation ou lemmatisation
    - **modèle avancé Deep Learning** :
        - mettre en minuscule (si ce n'est pas déjà fait)
        - retirer la ponctuation
        - retirer les espaces en trop
        - normaliser via stemmatisation ou lemmatisation

            Ces opérations sont plus légères que pour le modèle simple, car nous utilisons pour le modèle avancé un *LSTM*, un réseau de neurones récurrents capable de prendre en compte les **relations de dépendance entre les éléments successifs d'une séquence**. Il est donc par exemple important de conserver les stop words car le réseau sera capable de détecter la différence entre *happy* et *not happy*.

## IV) Les modèles

### IV.0) La démarche

#### IV.0.1) Séparation des données

De manière classique nous avons séparé nos données en un jeu d'entraînement `X_train, y_train`et un jeu de test `X_test, y_test`.

Pour comparer les modèles issus de la recherche des meilleurs hyperparamètres, nous avons utilisé :
- une **validation croisée** pour le **modèle simple** 
- un split **train --> train/validation** pour le **modèle avancé** (eu égard aux temps d'entraînement bien plus conséquents)

#### IV.0.2) Quelle métrique d'évaluation ?

*Air Paradis* cherche à anticiper les bad buzz, **la classe qui nous intéresse est donc *1 - Negative*** (pour rappel nous avons ré-attribué les étiquettes : 0 --> 1 et 4 --> 0). 

Nous avons la chance d'avoir un jeu de données **équilibré**, nous permettant de rester classique sur le choix de la métrique d'évaluation :

- Nous pourrions nous intéresser seulement à l'*accuracy*, mais cela reviendrait à choisir à la place du client un seuil de décision égale 0.5.
- Nous allons donc **chercher à optimiser la *ROC AUC***, qui nous permettra dans cette première phase du projet de sélectionner le modèle le plus performant tout en restant polyvalent. *Air Paradis* pourra par la suite travailler avec nous pour déterminer le seuil le plus pertinent pour son besoin.

Nous avons **enregistré** également l'*accuracy* et le *training time*, et avons **tracé** les courbes *ROC* et *Precision-Recall*.

### IV.1) Modèle classique de machine learning - Régression Logistique

Pour le modèle simple, nous avons :
- créé un **pipeline d'apprentissage** pour un modèle scikit learn dans le cadre d'une validation croisée, avec le tracking offert par `MLFlow`
- créé des classes `Scikit Learn` intégrant toutes les opération de nettoyages, afin de pouvoir êtres intégrées dans une `pipeline` `Scikit Learn`
- utilisé le `TfidfVectorizer` pour représenter notre corpus de façon numérique et utilisant la fréquence des mots à l'échelle des documents, **mais aussi à l'échelle du corpus**, afin de **réduire l'impact des mots trop fréquents**
- créé une fonction de recherche d'hyperparamètres basée sur `Optuna`

#### V.1.1) *Dummy*

<img src="./mySaves/plots/simple_0_dummy.png" alt="simple_0_dummy" width="800" class="center"/>

#### V.1.2) *Logistic regression* - Premier essai

<img src="./mySaves/plots/simple_1_first_try.png" alt="simple_1_first_try" width="800" class="center"/>

#### V.1.3) *Logistic regression* - Optuna

Utilisation d'`Optuna` pour trouver les meilleurs hyperparamètres :
- pour le `simpleModelCleaner` 
- puis pour `LogisticRegression`

<img src="./mySaves/plots/simple_2_optuna_clean.png" alt="simple_2_optuna_clean" width="800" class="center"/>
<img src="./mySaves/plots/simple_3_optuna_LR.png" alt="simple_3_optuna_LR" width="800" class="center"/>

#### V.1.4) *Logistic regression* - Modèle final

Suite aux différentes étapes réalisées pour la construction de notre modèle simple, nous pouvons apprécier l'évolution des résultats sur la CV :


<img src="./mySaves/plots/simple_4_bilan.png" alt="simple_4_bilan" width="800" class="center"/>

Nous pouvons ainsi contruire notre modèle simple final :

<img src="./mySaves/vscode_captures/simple_pipeline_sklearn.png" alt="simple_pipeline_sklearn" width="2000" class="center"/>

Et l'utiliser sur notre set de test :

<img src="./mySaves/plots/simple_5_final.png" alt="simple_5_final" width="800" class="center"/>

### IV.2) Modèle avancé de deep learning - Bidirectional LSTM

Pour le modèle avancé nous avons utilisé :
- Une couche `Text_vectorization` :
    - nettoyer préalablement nos tweets via l'argument `standardize` : nous permettant d'appliquer différentes combinaisons d'opérations (décrites ci-dessus dans *III) Preprocessing*)
    - transformer notre Series de tweets **en une liste de séquences d'indices de tokens** (via l'argument `output_mod = "int"`). Le nombre d'indices disponible est **borné par `max_tokens`, qui est la taille maximale** du dictionnaire
    - appliquer un *padding*, c'est-à-dire remplir les séquences avec des " "  (ou les tronquer si elles sont trop longues) afin qu'elles atteignent un longueur donnée (`output_sequence_length`).
- Une **couche d'embedding**, qui crée des plongements de mots :
    - Elle prend en entrée des **séquences d'entiers** issues de la vectorisation
    - Elle a pour arguments `input_dim` qui correspond au nombre de mots uniques dans le vocabulaire (déterminé lors de la vectorisation) ainsi que `output_dim` qui correspond à la dimension des vecteurs de mots
    - Elle sort alors un tensor de dimensions **nb de séquences $*$ longueur des séquence $*$ dimension des vecteurs de mot**
    - Nous pouvons initialiser les poids de cette couche d'embedding grâce un embedding **pré-entraîné**, pour cela nous avons **créé une fonction** permettant de créer cette matrice de poids pré-entraînés à partir d'un fichier téléchargé sur internet : `get_embedding_matrix`.
- Une **couche LSMT (Long Short-Term Memory) bidirectionnelle**, qui permet de dédoubler les unités *LSTM* afin de parcourir les séquences de gauche à droite (sens de la lecture) et de droite à gauche, permettant ainsi de **capter des dépendances dans les 2 sens**. Pour cette couche nous pouvons faire varier :
    - `units` : qui correspond au nombre d'unités *LSTM* dans chacune des 2 directions
    - `dropout` : qui correspond à la fraction d'unités *LSTM* qui seront déconnectées à chaque étape de la lecture des séquences
    - `recurrent_dropout` : qui correspond à la fraction d'unités *LSTM* qui se seront déconnectées à chaque étape de la lecture de la cellule mémoire
- une **dense** à 16 unités avec une fonction d'activation **relu** (pour ajouter de la non-linéarité)
- une **dense** à une seule unité (une seule classe à prédire) avec une fonction d'activation **sigmoïde** (pour obtenir un score/une probabilité entre 0 et 1)

La création et l'entraînement de ce modèle a été intégré à différents pipelines d'apprentissage, au sein de *runs* `MLFlow` pour le tracking.

#### V.2.1) *Baseline*

Nous avons utilisé un modèle **non récurrent** (pas de couche LSTM mais un Flatten) comme base de comparaison :

<img src="./mySaves/plots/advanced_0_baseline.png" alt="advanced_0_baseline" width="800" class="center"/>

#### V.2.2) Premier essai de notre model `LSTM` bidirectionnel

Nous avons d'abord testé notre modèle **sans normalisation** et **sans matrice d'embedding** pré-entraîné :

<img src="./mySaves/plots/advanced_1_first_try.png" alt="advanced_1_first_try" width="800" class="center"/>

#### V.2.3) **`normalization = "stem"`** ou **`normalization = "lem"`** ?

Puis nous avons testé la lemmatisation et la stemmatisation :

<img src="./mySaves/plots/advanced_2_stem.png" alt="advanced_2_stem" width="800" class="center"/>
<img src="./mySaves/plots/advanced_3_lem.png" alt="advanced_3_lem" width="800" class="center"/>

Nous avons choisi la lemmatisation.

#### V.2.4) *Quel embedding* ?

Nous avons ensuite comparé **GLOVE** et **FASTTEXT**, **entraînable** ou **pas** :

<img src="./mySaves/vscode_captures/advanced_embedding_results.png" alt="advanced_embedding_results" width="1000" class="center"/>

Nous avons choisi GLOVE avec une couche entraînable.

#### V.2.5) Quels hyperparamètres pour le modèle `LSTM` ?

Une fois choisi les hyperparamètres des étapes préliminaires de notre pipeline, nous nous sommes intéressés :
- au modèle `LSTM` (`LSTM_units`, `LSTM_dropout`, `LSTM_recurrent_dropout`, `optimizer`)
- aux paramètres d'entraînement (`early_stopping_patience`, `batch_size`)

<img src="./mySaves/mlflow_ui_plots/advanced/6_optuna_26_best.png" alt="6_optuna_26_best" width="2000" class="center"/>

#### V.2.6) Modèle avancé final

Suite aux différentes étapes réalisées pour la construction de notre modèle avancé, nous pouvons apprécier l'évolution des résultats sur le set de validation :


<img src="./mySaves/plots/advanced_9_bilan.png" alt="advanced_9_bilan" width="800" class="center"/>

Nous pouvons ainsi contruire notre modèle avancé final :

<img src="./mySaves/vscode_captures/advanced_final.png" alt="advanced_final" width="800" class="center"/>

Et l'utiliser sur notre set de test :

<img src="./mySaves/plots/advanced_10_final.png" alt="advanced_10_final" width="800" class="center"/>

### IV.3) Test de l'apport d'un modèle BERT ou d'un embedding USE

Dans le  cadre du projet, nous avons également testé des modèles basés sur **BERT** (*Bidirectional Encoder Representations from Transformers*) et sur **USE** (*Universal Sentence Encoder*), avec des résultats très intéressants en termes de **performance** :

<img src="./mySaves/plots/COMPARISON_ON_TEST.png" alt="COMPARISON_ON_TEST" width="800" class="center"/>

## V) Interprétabilité du modèle avancé

Nous avons étudier l'interprétabilité du modèle avancé afin de rendre moins opaque le processus de décision du modèle. Nous avons utilisé la libraire `SHAP` qui utilise des approximations des valeurs de Shapley et est une des méthodes d'explicabilité les plus utilisées. L'objectif est de connaître la contribution de chaque feature à une prédiction donnée, ou plus exactement : comment chaque feature fait différer une prédiction de la prédiction moyenne.


### V.1) Interprétabilité globale

<img src="./mySaves/shap/global_imp.png" alt="global_imp" width="1200" class="center"/>

### V.2) Interprétabilité locale

<img src="./mySaves/shap/force_plot_0.1.png" alt="force_plot_0.1" width="1000" class="center"/>
<img src="./mySaves/shap/force_plot_0.2.png" alt="force_plot_0.3" width="1000" class="center"/>
<img src="./mySaves/shap/force_plot_0.3.png" alt="force_plot_0.3" width="1000" class="center"/>
<img src="./mySaves/shap/force_plot_1.1.png" alt="force_plot_1.1" width="1000" class="center"/>
<img src="./mySaves/shap/force_plot_1.2.png" alt="force_plot_1.2" width="1000" class="center"/>
<img src="./mySaves/shap/force_plot_1.3.png" alt="force_plot_1.3" width="1000" class="center"/>

## VI) Conclusion

Nous avons pu explorer différentes approches de modélisation pour répondre à la problématique du projet. Nous avons optimisé deux modèles : Un modèle simple basé sur une régression linéaire et un modèle avancé basé sur un LSTM bidirectionnel. Des modèles basés sur BERT et USE ont également été testés. Ces derniers ont donné des résultats très prometteurs, sans avoir été optimisés. Explorer plus de modèles pré-trainés basés sur des transformers nous permttrait sans doute de gagner en performance, voir même en interprétabilité grâce à des librairies spécialisées comme `transformers_interpret`.